<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Надо построить модель со значением метрики качества *F1* не меньше 0.75. 

План работы:

1. Загрузим и подготовим данные:
 - лемматизация текста 
 - очистка текста
 - разобьем данные на обучающую , валидационную и  тестовую выборки.
2. Обучим разные модели:
 - логистическая регрессия
 - дерево решений
 - случайный лес
3. Выводы


Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# загрузка библиотек

import pandas as pd
import numpy as np
import nltk
import re 

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.featstruct import Feature
from nltk.corpus.reader.reviews import FEATURES
from nltk.corpus import wordnet
from nltk.tag import pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score

In [2]:
# загрузка данных
data = pd.read_csv('/datasets/toxic_comments.csv')
data.duplicated().sum()
display(data.head())
display(data.info())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

In [4]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
corpus = list(data['text'])

# функция для лемматизации текста

def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    
    m = WordNetLemmatizer()
    lemm_list = nltk.word_tokenize(text)
    lemm_list = [m.lemmatize(word, get_wordnet_pos(word)) for word in lemm_list]
    lemm_text = " ".join(lemm_list)
        
    return lemm_text

# функция для очистки текста

def clear_text(text):
    text = text.lower()
    clear_list = re.sub(r'[^a-zA-Z ]', ' ', text) 
    clear_text = clear_list.split()
    clear_text = " ".join(clear_text)
    
    return clear_text



[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
text_lemm = data['text'].apply(lambda x: lemmatize(clear_text(x))).values

text_lemm[:2]

array(['explanation why the edits make under my username hardcore metallica fan be revert they weren t vandalism just closure on some gas after i vote at new york doll fac and please don t remove the template from the talk page since i m retire now',
       'd aww he match this background colour i m seemingly stuck with thanks talk january utc'],
      dtype=object)

In [6]:
data['text_lemm'] = text_lemm
data.head()

,text,toxic,text_lemm
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [7]:
# загрузка множества стоп-слов английского языка
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

## Обучение

In [9]:
# разделим данные на обучающую, валидационную и тестовую выборки

train, test = train_test_split(data, test_size=0.2, random_state=12345)
train, valid = train_test_split(train, test_size=0.25, random_state=12345)

print(train.shape)
print(valid.shape)
print(test.shape)

(95742, 3)
(31914, 3)
(31915, 3)


In [10]:
train.head()

,text,toxic,text_lemm
3180,This person needs reporting to the Administrat...,0,this person need reporting to the administrato...
134113,Issues addressed \n\nI'm just going to outline...,0,issue address i m just go to outline the issue...
54472,Red Pepper Cases \n\nRed Pepper Cases\n\nRed P...,0,red pepper case red pepper case red pepper wat...
29333,"Alpha-Bits are back in 2008, They are being so...",0,alpha bit be back in they be be sell a of july...
13869,YOU DUMB ASS!!\nWhy the fuck would you delete ...,1,you dumb as why the fuck would you delete the ...


Создадим матрицу признаков для обучающей выборки cо значениями TF-IDF.

In [11]:
corpus_train = list(train['text_lemm'])

count_tf_idf = TfidfVectorizer(ngram_range=(1,2), stop_words=stopwords) 
tf_idf = count_tf_idf.fit_transform(corpus_train) 

target_train  = train['toxic']
features_train = tf_idf

In [12]:
target_train.value_counts()

0    86075
1     9667
Name: toxic, dtype: int64

В нашей задаче наблюдается сильный дисбаланс классов (9:1), что плохо сказывается на обучении модели. Классы не сбалансированны, когда их соотношение далеко от 1:1. Баланс классов наблюдается, если их количество примерно равно. Сбалансируем веса установив class_weight='balanced'.
Оценим качество моделей метрикой F1. 
F1 = 1  означает, что соотношение полноты и точности равно 1:1. 

### Логистическая регрессия

In [13]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted = model.predict(features_train)


In [14]:
corpus_valid = list(valid['text_lemm'])

tf_idf_valid = count_tf_idf.transform(corpus_valid) 
features_valid = tf_idf_valid
predicted_valid = model.predict(features_valid)
target_valid = valid['toxic']

In [15]:
# значение метрики F1
f1 = f1_score(target_valid, predicted_valid)
f1

0.7478945938603642

### Дерево решений

In [16]:
model_tree = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model_tree.fit(features_train, target_train)

predicted_valid = model_tree.predict(features_valid)
f1_tree = f1_score(target_valid, predicted_valid)


In [17]:
f1_tree

0.6494464944649447

### Случайный лес

In [18]:
model_random = RandomForestClassifier(random_state=12345, class_weight='balanced')
model_random.fit(features_train, target_train) 
predicted_valid = model_random.predict(features_valid)
f1_random = f1_score(target_valid, predicted_valid)

In [19]:
f1_random

0.5525075010715816

Представим полученные значения метрики F1 на разных моделях в виде таблицы

In [20]:
table = {'Модель' : ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия'], 
    'F1 ' : [f1_tree, f1_random, f1]
        }
        
frame = pd.DataFrame(table)
frame

,Модель,F1
0,Решающее дерево,0.649446
1,Случайный лес,0.552508
2,Логистическая регрессия,0.747895


Лучший результат получили на модели логистичекой регрессии, подберем для неё гиперпараметры

```parameters = {'C': np.linspace(10, 15),
             'max_iter': [1000]}```

```clf = GridSearchCV(model, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
clf.fit(features_train, target_train)```

```print(f"Наилучший показатель f1 на кросс-валидации : {clf.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf.best_params_}")```

Наилучший показатель f1 на кросс-валидации : 0.761
Параметр регуляризации для лучшей модели: {'C': 11.73469387755102, 'max_iter': 1000}

Необходимое значние метрики получили на модели логистической регрессии F1 = 0.761 при следующих параметрах: {'C': 11.73469387755102, 'max_iter': 1000}

## Тестирование модели

In [21]:
# тестирование модели логистической регресии
corpus_test = list(test['text_lemm'])
 
tf_idf_test = count_tf_idf.transform(corpus_test) 
features_test = tf_idf_test
target_test = test['toxic']

model_best = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced', C=11.73469387755102, max_iter=1000)
model_best.fit(features_train, target_train)
predicted = model_best.predict(features_test)
f1_lr = f1_score(target_test, predicted)
f1_lr

0.7827017114914426

## Выводы

Для того, чтобы предложить интернет-магазину инструмент, который будет искать токсичные комментарии и отправлять их на модерацию, решали задачу бинарной классификации на наборе данных с разметкой о токсичности правок.  
 Для того проведены следующие этапы:
  
  - Загрузили и подготовили данные:
      - лемматизировали текст
      - очистили текст
      - создали  матрицу признаков cо значениями TF-IDFпризнаки 
      - разбили данные на обучающую , валидационную и тестовую выборки.
  - Обучили разные модели:
      - логистическую регрессию
      - дерево решений
      - случайный лес

Необходимое значение метрики качества F1 получили на модели линейной регрессии 0,782.  
Интернет-магазину «Викишоп» рекомендуется использовать для класификации комментариев модель логистической регресии.